In [6]:
# pip install azure-ai-translation-text

import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.translation.text import TextTranslationClient
from azure.ai.translation.text.models import InputTextItem
from dotenv import load_dotenv
load_dotenv()

endpoint = os.environ["AZURE_AI_SERVICES_URL"]
key = os.environ["AZURE_AI_SERVICES_KEY"]

credential = AzureKeyCredential(key)
client = TextTranslationClient(credential=credential, endpoint=endpoint)

## Translate Text Input

In [7]:
inputText = "Hello world, how are you doing today?"

input_text_elements = [InputTextItem(text=inputText)]
translationResponse = client.translate(content=input_text_elements, to=['es', 'fr', 'de'])
translations = translationResponse if translationResponse else None
for translation in translations:
    sourceLanguage = translation.detected_language
    for translated_text in translation.translations:
        print(f"'{inputText}' was translated from {sourceLanguage.language} to {translated_text.to} as: '{translated_text.text}'.")

'Hello world, how are you doing today?' was translated from en to es as: 'Hola mundo, ¿cómo estáis hoy?'.
'Hello world, how are you doing today?' was translated from en to fr as: 'Bonjour tout le monde, comment allez-vous aujourd’hui ?'.
'Hello world, how are you doing today?' was translated from en to de as: 'Hallo Welt, wie geht es dir heute?'.


## Read From Image and Translate

In [8]:
import cv2
from azure.core.credentials import AzureKeyCredential
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures

image_path = "../images/SpanishSign.png"
with open(image_path, "rb") as image_file:
    file_bytes = image_file.read()

img_client = ImageAnalysisClient(endpoint=endpoint, credential=AzureKeyCredential(key))
result = img_client.analyze(
    image_data=file_bytes,
    visual_features=[VisualFeatures.READ],
)

image = cv2.imread(image_path)
words_to_translate = []
for block in result.read.blocks:
    for line in block.lines:
        line_box = line['boundingPolygon']
        top_left = (line_box[0]['x'], line_box[0]['y'])
        bottom_right = (line_box[2]['x'], line_box[2]['y'])
        cv2.rectangle(
            image,
            top_left,
            bottom_right,
            (0, 0, 255), 2
        )
        for word in line.words:
            word_box = word['boundingPolygon']
            top_left = (word_box[0]['x'], word_box[0]['y'])
            bottom_right = (word_box[2]['x'], word_box[2]['y'])
            cv2.rectangle(
                image,
                top_left,
                bottom_right,
                (0, 255, 0), 2
            )
            print(f"Word: {word['text']} Confidence: {word['confidence']}")
            words_to_translate.append(word['text'])

cv2.imshow('Image with Bounding Box', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

if words_to_translate:
    words = " ".join(words_to_translate)
    input_text_elements = [InputTextItem(text=words)]
    translationResponse = client.translate(content=input_text_elements, to=['en'])
    translation = translationResponse[0] if translationResponse else None
    if translation:
        sourceLanguage = translation.detected_language
        for translated_text in translation.translations:
            print(f"'{words}' was translated from {sourceLanguage.language} to {translated_text.to} as: '{translated_text.text}'.")

Word: PELIGRO Confidence: 0.985
Word: PERSONAL Confidence: 0.99
Word: SIN Confidence: 0.994
Word: AUTORIZACIÓN Confidence: 0.878
Word: MANTENERSE Confidence: 0.994
Word: FUERA Confidence: 0.998


2024-04-14 20:19:40.358 Python[82526:3252326] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


'PELIGRO PERSONAL SIN AUTORIZACIÓN MANTENERSE FUERA' was translated from es to en as: 'UNAUTHORIZED PERSONAL DANGER STAYING OUT'.


: 